In [1]:
# Importing relevant packages and establishing the Exadata connection

import sqlalchemy
import pandas as pd
import numpy as np
import getpass

password=getpass.getpass("Enter your password:")
engine = sqlalchemy.create_engine("oracle+cx_oracle://anantk[an_ct_ws38]:%s@exa_gbtukprdd_an" %(password))

Enter your password:········


In [128]:
import sknn

In [22]:
import subprocess
import os
from urllib import quote_plus as urlquote

proxy = "anantk[an_ct_ws38]/Sushant123@exa_gbtukprdd_an"

# cmd_str =   "xdatadump userid='" + str(proxy) \
#                 + "' sqlstmt='SELECT  TRANSACTION_FID,NEEDSTATE,1 as value FROM AK_REP_BSK_PROD_68_M a where channel_flag = 1  " \
#                 +  "' delimiter=^ header=y > main_trans_small.csv"

cmd_str =   "xdatadump userid='" + str(proxy) \
                + "' sqlstmt='SELECT  TRANSACTION_FID,NSMGP,1 AS VALUE  \
FROM AK_REP_BSK_PROD_71 A WHERE channel_flag = 1 AND basket_qty BETWEEN 15 AND 20 " \
                +  "' delimiter=^ header=y > main_trans_med2.csv"        

print cmd_str
subprocess.call(cmd_str, cwd='./', shell=True)

xdatadump userid='anantk[an_ct_ws38]/Sushant123@exa_gbtukprdd_an' sqlstmt='SELECT  TRANSACTION_FID,NSMGP,1 AS VALUE  FROM AK_REP_BSK_PROD_71 A WHERE channel_flag = 1 AND basket_qty BETWEEN 15 AND 20 ' delimiter=^ header=y > main_trans_med2.csv


0

In [127]:
# import time

df = pd.read_csv('main_trans_large3.csv',header=0, delimiter="^").groupby(['TRANSACTION_FID','NSMGP'])['VALUE'].max().unstack()
df

start_time = time.time()

L = pd.DataFrame()
for i in df.columns:
    Y = pd.DataFrame()
    F = df[df[i]==1.0]
    b = np.array(F.fillna(0))
    x = b.sum(axis = 0)
    Y = pd.DataFrame(x).transpose()
    L = pd.concat([L,Y])       

L.columns = df.columns

print("--- RUNTIME : %s seconds ---" % (time.time() - start_time))

L.to_csv('/nfs/science/shared/ipythonNotebooks/anantk/main_adj_med_2.csv')

# X = df.as_matrix()
# # X

# where_are_NaNs = np.isnan(X)
# X[where_are_NaNs] = 0
# X
# df
L

NSMGP,(HERBS & SPICES),1 Litre Sauces (ASIAN STORE CUPBOARD),10mth Cereal (BABY FOOD),10mth Meal Jar (BABY FOOD),12mth+ Cereal (BABY FOOD),4-6mth Cereal (BABY FOOD),4-6mth Dessert (BABY FOOD),4-6mth Meal Jar (BABY FOOD),7-9mth Cereal (BABY FOOD),7-9mth Dessert (BABY FOOD),...,Wraps (SANDWICHES),Yeast Spread (PRESERVES),Yellow Bean Sauce (RICE & COOKING SAUCES),Yoghurt (CHILLED ASIAN FOODS),Yorkshire Puddings (YORKSHIRE PUDDINGS),Zinc / Selenium / Magnesium (VITAMINS),breaded (FROZEN FISH),soothers (BABY ACCESSORIES),speciality rice (ASIAN RICE),stout (DISCOVERY BEER)
0,147.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0
0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,572.0,38.0,5.0,10.0,33.0,14.0,48.0,27.0,...,2.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,38.0,1398.0,1.0,4.0,119.0,36.0,32.0,135.0,...,15.0,4.0,0.0,2.0,9.0,2.0,0.0,0.0,0.0,0.0
0,0.0,0.0,5.0,1.0,41.0,0.0,3.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,10.0,4.0,0.0,459.0,49.0,72.0,20.0,10.0,...,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,33.0,119.0,3.0,49.0,2993.0,1032.0,90.0,158.0,...,41.0,5.0,0.0,7.0,19.0,1.0,0.0,1.0,0.0,0.0
0,0.0,0.0,14.0,36.0,0.0,72.0,1032.0,2463.0,61.0,81.0,...,26.0,5.0,0.0,0.0,24.0,0.0,0.0,1.0,0.0,0.0
0,0.0,0.0,48.0,32.0,2.0,20.0,90.0,61.0,799.0,58.0,...,4.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,27.0,135.0,0.0,10.0,158.0,81.0,58.0,1421.0,...,15.0,0.0,0.0,3.0,7.0,0.0,0.0,2.0,0.0,0.0


In [125]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
import time

print("Fitting the NMF model for main 40 item basket," 
      " n_features= 3...")

t0 = time.time()

nmf = NMF(n_components=6, random_state=1,
          alpha=0,verbose =5, l1_ratio=0.5,init = 'nndsvd',sparseness = 'data')
nmf.fit(X)

print("done in %0.3fs." % (time.time() - t0))


Fitting the NMF model for main 40 item basket, n_features= 3...


/usr/lib64/python2.7/site-packages/sklearn/decomposition/nmf.py:996: DeprecationWarning: Controlling regularization through the sparseness, beta and eta arguments is only available for 'pg' solver, which will be removed in release 0.19. Use another solver with L1 or L2 regularization instead.
  " regularization instead.", DeprecationWarning)


('violation:', 1.0)
('violation:', 0.4083245820013992)
('violation:', 0.2824740931611489)
('violation:', 0.20211543937303697)
('violation:', 0.15888269790418158)
('violation:', 0.12839155921144021)
('violation:', 0.10890086211534244)
('violation:', 0.09577331427544164)
('violation:', 0.08700711030272648)
('violation:', 0.08010320494095952)
('violation:', 0.07413596216504229)
('violation:', 0.07055741961490858)
('violation:', 0.06718174547946153)
('violation:', 0.06322281004629714)
('violation:', 0.05888380614428537)
('violation:', 0.05409600167839554)
('violation:', 0.04811817933502915)
('violation:', 0.042371645831002835)
('violation:', 0.03804326287508571)
('violation:', 0.03516496457185654)
('violation:', 0.03284956748029489)
('violation:', 0.031137860631334124)
('violation:', 0.02981335978326155)
('violation:', 0.02877812326857142)
('violation:', 0.02746807387518527)
('violation:', 0.026772709576252073)
('violation:', 0.026194269844382084)
('violation:', 0.025799989897206683)
('vio

In [124]:
Y = pd.DataFrame(nmf.components_)
Y.to_csv('nmf3.csv')

In [21]:
cmd_str =   "xdatadump userid='" + str(proxy) \
                + "' sqlstmt='SELECT  DISTINCT A.NSMGP,NS_CODE,TPNB,BPEN  FROM \
(SELECT A.*,ROW_NUMBER()OVER(PARTITION BY CHANNEL_FLAG, NSMGP ORDER BY BPEN DESC) AS RANK \
FROM AK_REP_BSK_PROD_3 A) a \
INNER JOIN (SELECT DISTINCT NSMGP,NS_CODE,CHANNEL_FLAG FROM AK_REP_BSK_PROD_71 WHERE CHANNEL_FLAG =1)B \
ON A.NSMGP = B.NSMGP \
where A.channel_flag = 1 and rank = 1 order by ns_code" \
                +  "' delimiter=^ header=y > bpen_prod_main.csv"
print cmd_str
subprocess.call(cmd_str, cwd='./', shell=True)

xdatadump userid='anantk[an_ct_ws38]/Sushant123@exa_gbtukprdd_an' sqlstmt='SELECT  DISTINCT A.NSMGP,NS_CODE,TPNB,BPEN  FROM (SELECT A.*,ROW_NUMBER()OVER(PARTITION BY CHANNEL_FLAG, NSMGP ORDER BY BPEN DESC) AS RANK FROM AK_REP_BSK_PROD_3 A) a INNER JOIN (SELECT DISTINCT NSMGP,NS_CODE,CHANNEL_FLAG FROM AK_REP_BSK_PROD_71 WHERE CHANNEL_FLAG =1)B ON A.NSMGP = B.NSMGP where A.channel_flag = 1 and rank = 1 order by ns_code' delimiter=^ header=y > bpen_prod_main.csv


0

In [64]:
cmd_str =   "xdatadump userid='" + str(proxy) \
                + "' sqlstmt='SELECT DISTINCT NSMGP,NS_CODE,CHANNEL_FLAG,CATEGORY FROM AK_REP_BSK_PROD_71 A \
INNER JOIN AN_CT_WS46.PRODUCT_NEEDSTATES B ON A.NSMGP = B.NEEDSTATE_MERCHGROUP \
WHERE A.CHANNEL_FLAG =1 AND A.basket_qty BETWEEN 35 AND 40 order by ns_code " \
                +  "' delimiter=^ header=y > top_prod_main_med.csv"
print cmd_str
subprocess.call(cmd_str, cwd='./', shell=True)

xdatadump userid='anantk[an_ct_ws38]/Sushant123@exa_gbtukprdd_an' sqlstmt='SELECT DISTINCT NSMGP,NS_CODE,CHANNEL_FLAG,CATEGORY FROM AK_REP_BSK_PROD_71 A INNER JOIN AN_CT_WS46.PRODUCT_NEEDSTATES B ON A.NSMGP = B.NEEDSTATE_MERCHGROUP WHERE A.CHANNEL_FLAG =1 AND A.basket_qty BETWEEN 35 AND 40 order by ns_code ' delimiter=^ header=y > top_prod_main_med.csv


0

In [65]:
ns_2way = pd.read_csv("main_adj_large_2.csv")
bpen = pd.read_csv("bpen_prod_main.csv",header=0, delimiter="^")
prod = pd.read_csv("top_prod_main_med.csv",header=0, delimiter="^")

print ns_2way.shape
print ns_2way.head()
print prod.head()
print prod.shape
print bpen.head()

# bpen_matrix = bpen.iloc[:,3].as_matrix()
# print bpen_matrix.shape
# print bpen_matrix

Asso_matrix = ns_2way.ix[:,1:1740].as_matrix()

print Asso_matrix.shape
print Asso_matrix

np.fill_diagonal(Asso_matrix,0)
Asso_matrix

(1723, 1724)
   Unnamed: 0   (HERBS & SPICES)  1 Litre Sauces (ASIAN STORE CUPBOARD)  \
0           0              116.0                                    0.0   
1           0                0.0                                   49.0   
2           0                0.0                                    0.0   
3           0                0.0                                    0.0   
4           0                0.0                                    0.0   

   10mth Cereal (BABY FOOD)  10mth Meal Jar (BABY FOOD)  \
0                       0.0                         0.0   
1                       0.0                         0.0   
2                     258.0                        27.0   
3                      27.0                       547.0   
4                       2.0                         1.0   

   12mth+ Cereal (BABY FOOD)  4-6mth Cereal (BABY FOOD)  \
0                        0.0                        0.0   
1                        0.0                        0.0   
2   

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [66]:
col = ns_2way.ix[:,1:].columns
col1 = pd.DataFrame(col)
col1
prod

,NSMGP,NS_CODE,CHANNEL_FLAG,CATEGORY
0,(HERBS & SPICES),1,1,GROCER Y.
1,1 Litre Sauces (ASIAN STORE CUPBOARD),2,1,GROCER Y.
2,10mth Cereal (BABY FOOD),3,1,HB&WELLNES S.
3,10mth Meal Jar (BABY FOOD),4,1,HB&WELLNES S.
4,12mth+ Cereal (BABY FOOD),5,1,HB&WELLNES S.
5,4-6mth Cereal (BABY FOOD),6,1,HB&WELLNES S.
6,4-6mth Dessert (BABY FOOD),7,1,HB&WELLNES S.
7,4-6mth Meal Jar (BABY FOOD),8,1,HB&WELLNES S.
8,7-9mth Cereal (BABY FOOD),9,1,HB&WELLNES S.
9,7-9mth Dessert (BABY FOOD),10,1,HB&WELLNES S.


In [57]:
# x =  np.array([0,0,0,1,0,0,0,0,0,0,1,1,1])

# print x

# b = x - 1   

# print "\n",b

# print "\n",(x)*b

# print "\n",np.sum((x)*b)

# Y = prod.iloc[r]
# np.count(np.array(Y['CATEGORY'])

prod.iloc[x]['CATEGORY'].nunique()      

/usr/lib64/python2.7/site-packages/pandas/indexes/base.py:1275: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
  result = getitem(key)


2

In [59]:
from scipy.optimize import minimize

# Decision variables
x0 = np.array([0]*1723)
# x0 = res.x
print x0.shape
print x0

(1723,)
[0 0 0 ..., 0 0 0]


In [93]:
def obj (x):
    return (-1 * (np.sum((x*(np.array([x]).T))*Asso_matrix))) 
#             + 1000000000*((np.sum(x*(1-x)))**2)) 
#             + 10000000000*((10 - np.sum(x)**2)))

def cons2(x):
    'sum of x is equal to 10'
    return np.sum(x) - 41

def cons1(x):
    'sum of x is equal to 10'
    r = x.argsort()[-40:]
    Y = prod.iloc[r]
    return Y['CATEGORY'].nunique() - 13

# def cons2(x):
#     'sum of x is equal to 10'
#     return -1 * (np.sum((x)*bpen_matrix)

cons = ({'type': 'eq', 'fun': cons2},{'type': 'ineq', 'fun': cons1})

bnds = [[0,1] for i in range(1723)]

# print bnds

In [94]:
import time
start_time = time.time()

res = minimize(obj, x0, method='SLSQP',  bounds=bnds, constraints= cons, options={'disp': True,'maxiter' :200})
# res = minimize(obj, x0, method='L-BFGS-B',   bounds=bnds, options={'disp': True})
# constraints= cons,'iprint': 1,'maxiter' :1000 
# ,  bounds=bnds

delta_t = time.time() - start_time

print("--- RUNTIME : %s seconds ---" % (time.time() - start_time))

More than 3*n iterations in LSQ subproblem    (Exit mode 3)
            Current function value: -26779049.909
            Iterations: 6
            Function evaluations: 10380
            Gradient evaluations: 6
--- RUNTIME : 1079.20318484 seconds ---


In [101]:
# np.sum(res.x*(res.x - 1))
res
# delta_t
# np.sum(res.x)
# m = np.sort(res.x)
# print m[:-42]
# r = x.argsort()[:-11]
# r = (res.x).argsort()[-40:]
# # r
# Y = prod.iloc[r]
# Y
# r = np.array([77,216,270,278,382,475,727,836,1126,1186])
# col1.iloc[r]
# Y = pd.DataFrame(res.x)
# bpen.to_csv('/nfs/science/shared/ipythonNotebooks/anantk/ebpen.csv')  
# Y.to_csv('/nfs/science/shared/ipythonNotebooks/anantk/main_large_cons2.csv')  

     fun: -17771060.403414816
     jac: array([-1674.  ,  -540.75, -2967.75, ...,  -106.  ,   -24.  ,     0.  ])
 message: 'More than 3*n iterations in LSQ subproblem'
    nfev: 6910
     nit: 4
    njev: 4
  status: 3
 success: False
       x: array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [ ]:
As_matrix = ns_2way.ix[:,1:1208].as_matrix()

J = np.diag(As_matrix)
K = pd.DataFrame(J)
K.to_csv('/nfs/science/shared/ipythonNotebooks/anantk/J.csv')  

In [106]:
import pulp

ImportError: No module named pulp